In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]= "2,3"

In [2]:
import torch
model = torch.hub.load("pytorch/vision", "vit_b_16")

import torchvision
from Models.transformer import VisionTransformer as vit
import Models.Conv as conv

from DataLoader import CIFAR100
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn

import utils

import numpy as np
import torch.backends.cudnn as cudnn
import random

Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/root/.cache/torch/hub/pytorch_vision_main/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrain

In [3]:
train_loader, test_loader = CIFAR100.get_data(58*3)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
teacher = vit(class_num = 100, pretrained = True)
teacher.load_state_dict(torch.load("saved_models/vit_b_16_m_0_9_std_0_4_acc_89_01.pth").module.state_dict())
student = vit(class_num = 100, pretrained = True)

In [5]:
device = "cuda"

teacher = teacher.to(device)
teacher = torch.nn.DataParallel(teacher, device_ids=[0, 1])

student = student.to(device)
student = torch.nn.DataParallel(student, device_ids=[0, 1])


In [6]:
criterion_onlylabel = lambda a,b : mse(a*b, b)
criterion_CE = nn.CrossEntropyLoss()
mse = nn.MSELoss()
softmax = torch.nn.Softmax(dim = 1)
criterion_KLD = torch.nn.KLDivLoss(reduction="batchmean")
criterion_response = lambda a,b : criterion_KLD(torch.log_softmax(a, dim=1),torch.softmax(b, dim=1))


In [7]:
S_optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)
T_optimizer = optim.SGD(teacher.parameters(), lr=0.01, momentum=0.9)
CE_loss = nn.CrossEntropyLoss()

In [8]:
S_scheduler = torch.optim.lr_scheduler.MultiStepLR(S_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)
T_scheduler = torch.optim.lr_scheduler.MultiStepLR(T_optimizer, milestones=[1,2,3,4,5,6,7], gamma=0.1)

In [9]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)
best_acc = 0.0
stack = 0

accs_train = []
accs_test = []

In [10]:
utils.test(teacher, test_loader,device) # student도 변하는거 확인 완료함


100%|██████████| 58/58 [00:16<00:00,  3.47it/s]

0 	 test acc : 0.8901000022888184


tensor(0.8901, device='cuda:0')

In [ ]:
# 1. output에 대해 softmax지워봄

In [11]:
student_test_accs = []

try:
    encoder_length = len(teacher.encoder.layers)
except:
    encoder_length = len(teacher.module.encoder.layers)

for epoch in range(100):
    
    print(f"lr : {S_scheduler.get_last_lr()}")
    T_correct = 0
    S_correct = 0
    all_data = 0
    
    loss_distill = []
    loss_CE = []
    loss_response = []
    student.train()
    teacher.eval()
    for img, label in tqdm(train_loader):
        input_data = img.to(device)
        label = label.to(device)
        
        
        all_data += len(input_data)
        input_lrp = utils.get_LRP_img(input_data, label, teacher, criterion_CE, T_optimizer, mean=1.5, std = 0.1, mult = 0.4).cuda()
        
        S_optimizer.zero_grad()
        T_optimizer.zero_grad()

        layer = random.randint(0,  3+encoder_length)
        output_s, fk = student(input_data,layer)
        output_t, fk_lrp = teacher(input_lrp,layer)
        
        distill_loss = mse(fk, fk_lrp)
                    
        CE_loss = criterion_CE(output_s, label)
        
        response_loss = criterion_response(output_s, output_t)
        
        T_correct += sum(label == torch.argmax(output_t, dim=1))
        S_correct += sum(label == torch.argmax(output_s, dim=1))
        
        loss_CE.append(CE_loss.item())
        loss_distill.append(distill_loss.item())
        loss_response.append(response_loss.item())
        
        loss = (distill_loss * 2 + CE_loss) / 2
        loss.backward()
        S_optimizer.step()

    print("distill loss : ", sum(loss_distill) / len(loss_distill))
    print("general loss : ", sum(loss_CE) / len(loss_CE))
    print("response loss : ", sum(loss_response) / len(loss_response))
    
    print(f"Teacher acc: {T_correct / all_data}")
    print(f"Student acc: {S_correct / all_data}")

    test_acc = utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함
    
    if test_acc > best_acc + 0.01:
        stack = 0
        best_acc = test_acc
        
    else:
        stack+=1
    
    if stack > 3:  
        S_scheduler.step()
        stack = 0
        
    student_test_accs.append(test_acc.item())
    print("=" * 100)

lr : [0.01]


100%|██████████| 288/288 [05:54<00:00,  1.23s/it]


distill loss :  0.09352837664644337
general loss :  2.0429506248070135
response loss :  1.8423182517290115
Teacher acc: 0.9752999544143677
Student acc: 0.5237599611282349


100%|██████████| 58/58 [00:11<00:00,  4.91it/s]


0 	 test acc : 0.7893999814987183
lr : [0.01]


...

distill loss :  0.019297740679273073
general loss :  0.4610011844067938
response loss :  0.27668468359236914
Teacher acc: 0.973099946975708
Student acc: 0.8739799857139587


100%|██████████| 58/58 [00:11<00:00,  5.08it/s]


82 	 test acc : 0.8895999789237976
lr : [1.0000000000000005e-09]


 59%|█████▉    | 170/288 [03:31<02:26,  1.25s/it]


KeyboardInterrupt: 

In [ ]:
# distill loss를 2배 키워보는것도 좋을지도

In [ ]:
utils.test(teacher, test_loader,device, epoch) # student도 변하는거 확인 완료함
utils.test(student, test_loader,device, epoch) # student도 변하는거 확인 완료함


In [ ]:
torch.save(student, "vit_b_16_m_0_9_std_0_4_acc_89_01.pth")

In [ ]:
0 == "0"